In [4]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

# ----------------------------
# Load your CSV
# ----------------------------
csv_path = r'C:\Users\DELL\Desktop\Sehmilo\GitHub Repo\geominerAI_GeoGPT\data\Pitting_XRF.csv'
df = pd.read_csv(csv_path)

In [5]:
df.head()

,hole_id,lat,lon,elevation,depth,no_of_litho,INTERVAL,SiO2,V2O5,Cr2O3,...,TiO2,ZnO,Ag2O,Cl,ZrO2,SnO2,PbO,Rb2O,Cs2O,SrO
0,EST-001,10.572806,9.005417,849,2.5,4,A,60.26,0.11,0.04,...,1.97,0.03,1.95,0.59,0.50,0.02,0.03,0.06,0.08,0.04
1,EST-001,10.572806,9.005417,849,2.5,4,B,61.77,0.13,0.04,...,2.60,0.03,0.06,0.48,0.31,0.00,0.02,0.05,0.06,0.03
2,EST-001,10.572806,9.005417,849,2.5,4,C,64.79,0.12,0.05,...,2.28,0.02,0.02,0.63,0.42,0.01,0.02,0.05,0.05,0.03
3,EST-001,10.572806,9.005417,849,2.5,4,D,61.73,0.12,0.02,...,2.43,0.04,0.07,0.56,0.41,0.00,0.03,0.05,0.04,0.03
4,EST-002,10.573194,9.005500,845,2.2,4,A,66.93,0.11,0.03,...,2.00,0.02,NaN,0.48,0.51,0.10,0.02,0.04,0.05,0.02


In [6]:


# ----------------------------
# Interval rules (depth below ground surface, meters)
# ----------------------------
interval_bounds = {
    "A": (0.0, 0.3),
    "B": (0.3, 1.0),
    "C": (1.0, 2.0),
    "D": (2.0, None),  # None means "to pit bottom"
}

# ----------------------------
# Build From/To using INTERVAL + depth
# ----------------------------
def compute_from_to(row):
    interval = str(row["INTERVAL"]).strip().upper()
    pit_depth = float(row["depth"])

    if interval not in interval_bounds or np.isnan(pit_depth):
        return np.nan, np.nan

    d_from, d_to = interval_bounds[interval]
    if d_to is None:
        d_to = pit_depth

    # Clamp to [0, pit_depth]
    d_from = max(0.0, min(d_from, pit_depth))
    d_to = max(0.0, min(d_to, pit_depth))

    # If interval doesn't exist for that pit depth, mark invalid
    if d_to <= d_from:
        return np.nan, np.nan

    return d_from, d_to

df[["Depth_From", "Depth_To"]] = df.apply(
    lambda r: pd.Series(compute_from_to(r)),
    axis=1
)

# Keep only valid intervals
df = df.dropna(subset=["Depth_From", "Depth_To"]).copy()

# ----------------------------
# Convert to absolute Z using elevation
# Z_top = surface elevation - depth_from
# Z_bottom = surface elevation - depth_to
# ----------------------------
df["Z_top"] = df["elevation"] - df["Depth_From"]
df["Z_bottom"] = df["elevation"] - df["Depth_To"]

# Optional: choose a grade column for hover (SnO2 exists in your file)
grade_col = "SnO2" if "SnO2" in df.columns else None

# ----------------------------
# Create interactive 3D figure
# Each interval is a vertical segment at same (lon, lat)
# ----------------------------
fig = go.Figure()

# A consistent interval order helps legend readability
interval_order = ["A", "B", "C", "D"]
palette = px.colors.qualitative.Set2
color_map = {k: palette[i % len(palette)] for i, k in enumerate(interval_order)}

for interval in interval_order:
    dfi = df[df["INTERVAL"].astype(str).str.upper() == interval]
    if dfi.empty:
        continue

    for _, row in dfi.iterrows():
        hover = (
            f"Hole: {row['hole_id']}<br>"
            f"Interval: {interval}<br>"
            f"From: {row['Depth_From']} m<br>"
            f"To: {row['Depth_To']} m<br>"
            f"Pit depth: {row['depth']} m<br>"
            f"Elevation: {row['elevation']} m"
        )
        if grade_col:
            hover += f"<br>{grade_col}: {row[grade_col]}"

        fig.add_trace(
            go.Scatter3d(
                x=[row["lon"], row["lon"]],
                y=[row["lat"], row["lat"]],
                z=[row["Z_top"], row["Z_bottom"]],
                mode="lines",
                line=dict(width=10, color=color_map.get(interval, None)),
                name=f"Interval {interval}",
                showlegend=False,  # we will add a clean legend manually below
                hovertemplate=hover + "<extra></extra>",
            )
        )

# Add clean legend entries (one per interval)
for interval in interval_order:
    fig.add_trace(
        go.Scatter3d(
            x=[None], y=[None], z=[None],
            mode="lines",
            line=dict(width=10, color=color_map.get(interval, None)),
            name=f"Interval {interval}"
        )
    )

fig.update_layout(
    title="Pit Interval 3D Web Scene (stacked by depth)",
    scene=dict(
        xaxis_title="Longitude",
        yaxis_title="Latitude",
        zaxis_title="Elevation (m)",
        aspectmode="data",
    ),
    margin=dict(l=0, r=0, b=0, t=40),
    legend=dict(itemsizing="constant")
)

fig.show()

# ----------------------------
# Export as a standalone interactive web scene (HTML)
# ----------------------------
out_html = "pit_intervals_3d.html"
fig.write_html(out_html, include_plotlyjs="cdn")
print(f"Saved interactive scene to: {out_html}")


Saved interactive scene to: pit_intervals_3d.html
